# Summary:

#### ...

#### In particular, the following questions will be analyzed:
#### - How does...?



## ToDo:
#### choose continuous function as target first. then add noise to it and magnify it. 
#### also test: add noise ontop of sine or as another feature next to it - should give same answer?
#### also check: keras.layers.RNN

# Table of contents
* [1. update](#Part1_link)
* [2. update](#Part2_link)
<br >&nbsp;&nbsp;&nbsp;[2.1 update](#Part2.1_link)
<br >&nbsp;&nbsp;&nbsp;[2.2 update](#Part2.2_link)

* [3. update](#Part3_link)
<br >&nbsp;&nbsp;&nbsp;[3.1 update](#Part3.1_link)
* [4. update](#Part4_link)
<br >&nbsp;&nbsp;&nbsp;[4.1 update](#Part4.1_link)
* [5. update](#Part5_link)
<br >&nbsp;&nbsp;&nbsp;[5.1 update](#Part5.1_link)
* [6. update](#Part6_link)
<br >&nbsp;&nbsp;&nbsp;[6.1 update](#Part6.1_link)
* [7. update](#Part7_link)
<br >&nbsp;&nbsp;&nbsp;[7.1 update](#Part7.1_link)
<br >&nbsp;&nbsp;&nbsp;[7.2 update](#Part7.2_link)

<a id='Part1_link'></a>
# 1. Load modules

In [1]:
import Kind_of_Blue  # own class with a collection of methods used in this analysis

import tensorflow as tf

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

import numpy as np
import pandas as pd


<a id='Part2_link'></a>
# 2. Setup data

<a id='Part2.1_link'></a>
### 2.1 Generate time series: sine wave

The dataset used in this initial experiment is a simple sine wave. The number of data points in each period is kept fixed thoughout the analysis.

In [2]:
# set a range of dates on which the observations are made
idx = pd.date_range(end='7/1/2020', periods=10*364, freq='d')

# take a sine function as the observations
num_periods = 20  # number of sine periods
observations = [np.sin(2*np.pi*num_periods*x/len(idx)) for x in range(len(idx))]
print('number of observations in time series: {}'.format(len(observations)))

# initialize dataframe to store time series
df = pd.DataFrame(data=observations, columns=['observations'])
df.index = idx

# initialize object
mdq = Kind_of_Blue.Kind_of_Blue()

# load dataframe into object
mdq._selected_features = ['observations']
mdq.df = df
mdq.df.head()

number of observations in time series: 3640


,observations
2010-07-15,0.000000
2010-07-16,0.034516
2010-07-17,0.068991
2010-07-18,0.103384
2010-07-19,0.137654


<a id='Part2.2_link'></a>
### 2.2 Separate training and validation data 

In [3]:
# train-validation split ratio as class attribute set to 70%
print('train split ratio = ', mdq.TRAIN_SPLIT_RATIO)

train split ratio =  0.7


In [4]:
# initialize dataset from dataframe 
mdq.initialize_dataset()
print('loaded data set length: {}'.format(len(mdq._dataset)))

loaded data set length: 3640


<a id='Part3_link'></a>
# 3. Feature scaling

<a id='Part3.1_link'></a>
### 3.1 Standardize training data

In [5]:
# standardize data
mdq.standardize_data()

# check that mean equals zero and the standard deviation is one
print('mean: {}, std: {}'.format(round(np.mean(mdq._dataset), 2), round(np.std(mdq._dataset), 2)))

mean: 0.0, std: 1.0


<a id='Part4_link'></a>
# 4. Preprocess data

<a id='Part4.1_link'></a>
### 4.1 Cache, batch and shuffle using Tensorflow's data format

The number time points to be forecasted and the number of points the model is trained on are set. The training and validation dataset is generated.

In [6]:
# set number of time points for 1/ future forecasting points and 2/ the past, historical time points
future_target_size = int(365/52)
past_history_size = int(2*365)

# set batch size
batch_size = 32
mdq.generate_train_and_val_data(future_target_size=future_target_size, past_history_size=past_history_size
                                , batch_size=batch_size)

print('number of training samples: {}'.format(mdq._num_samples))

training set shape: x:(1818, 730, 1), y:(1818, 7, 1)
validation set shape: x:(355, 730, 1), y:(355, 7, 1)
number of training samples: 1818


<a id='Part5_link'></a>
# 5. Setup and compile model

<a id='Part5.1_link'></a>
### 5.1 LSTM 

Using Keras, the configuration details of the LSTM model is set and compiled.

In [7]:
# setup Keras LSTM model and compile
units = 32
num_layers = 3
mdq.compile_LSTM_model(units=units, num_layers=num_layers)

# check that model exists in mdq-object
print('test that model got created in object instance: \n', mdq._model)

ToDo2: change a few things in the LSTM setup before running tests
test that model got created in object instance: 


<a id='Part6_link'></a>
# 6. Fit model

<a id='Part6.1_link'></a>
### 6.1 LSTM 

Setting the number of epochs and steps per epoch, the model is fit to the training data. The generated 'history' will be used in the following section to assess the quality of the fit.

In [ ]:
# set number of epochs
epochs = 5

# set number of 
num_samples = mdq._num_samples
steps_per_epoch = int(num_samples/batch_size)

validation_steps = int(steps_per_epoch/2)

mdq.fit_model(epochs=epochs, steps_per_epoch=steps_per_epoch
              ,validation_steps=validation_steps)



Epoch 1/5
56/56 [==============================] - 45s 796ms/step - loss: 0.5335 - mse: 0.5335 - val_loss: 0.0890 - val_mse: 0.0890
Epoch 2/5
56/56 [==============================] - 41s 734ms/step - loss: 0.1091 - mse: 0.1091 - val_loss: 0.0188 - val_mse: 0.0188
Epoch 3/5
56/56 [==============================] - 40s 713ms/step - loss: 0.0762 - mse: 0.0762 - val_loss: 0.0045 - val_mse: 0.0045
Epoch 4/5
12/56 [=====>........................] - ETA: 28s - loss: 0.0656 - mse: 0.0656

<a id='Part7_link'></a>
# 7. loss and error metrics

<a id='Part7.1_link'></a>
### 7.1 Visualize training and validation history

LSTM model

<a id='Part7.2_link'></a>
### 7.2 Assess error metrics

In [ ]:
future_target_size

In [ ]:
mdq.plot_history()

LSTM model

In [ ]:
# visualize predictions

num = 2
for x, y in mdq._val_data.take(3):
    mdq.multi_step_plot(x[num], y[num], mdq._model.predict(x)[:1])